# Matrix Factorization

<b>User-based</b> и <b>Item-based</b> методы фильтрации страдают от <i>data sparsity</i> и <i>scalability</i>, из-за этого мы точно не можем рекомендовать очень хорошо. 

<b>MF</b> помогает решить это из-за возможности уменьшения размерности матрицы рейтингов.

Пример: в нашем наборе MovieLens 100k есть $m=610$ пользователей для $n=9724$ элементов. В основном, пользователи коммуницируют менее чем с $1\%$ элементов, матрица получается очень разреженной. Пример:

\begin{equation}
sparsity = 100 - \frac{\text{total # ratings}}{m \times n} = 100 - \frac{100000}{610\times 9724} = 98,3\%
\end{equation}

Matrix Factorization (MF) описана здесь <a href='https://ieeexplore.ieee.org/document/5197422'>(Yehuda Koren et al., 2009)</a>. MF декомпозицирут матрицу $R$ в 2 матрицы ( SVD создавало 3):

\begin{equation}
R = Q^\top P
\end{equation}

$P \in \mathbb{R}^{m\times k}$ латентные факторы пользователя, а  $Q \in \mathbb{R}^{n\times k}$ латентные факторы элементов. Каждая строка $P$, сообщает $p_u \in \mathbb{R}^k$ потребность (вкус в рейтинге) по каждому пользователю $u$ и каждая $q_i \in \mathbb{R}^k$ является фичей элемента $i$. Пересечение между $p_u$ и $q_i$ является предсказанным рейтингом для пользователя $u$ по элементу $i$ :

\begin{equation}
\hat{r}_{u,i} = q_{i}^{\top} p_u.
\end{equation}

<img src="img/MF.png">
<br>


## Matrix Factorization : алгоритм
<ol>
    <li>Инициализация $P$ и $Q$ с рандомными значениями
    <li>Для каждого примера $(u,i)\in\kappa$ выставить рейтинг $r_{u,i}$ :
        <ul>
            <li>вычислить  $\hat{r}_{u,i} = q_{i}^{\top} p_u$
            <li>вычислить ошибку : $e_{u,i} = |r_{ui} - \hat{r}_{u,i}|$
            <li>обновить $p_u$ и $q_i$:
                <ul>
                    <li>$p_u \leftarrow p_u + \alpha\cdot (e_{u,i}\cdot q_i-\lambda \cdot p_u)$
                    <li>$q_i \leftarrow q_i + \alpha\cdot (e_{u,i}\cdot p_u-\lambda \cdot q_i)$
                </ul>
        </ul>
    <li> повторять до подбора оптимальных параметров
</ol>


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

In [2]:
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def ttsplit(examples, labels, test_size=0.1, verbose=0):
    from sklearn.model_selection import train_test_split 
    
    if verbose:
        print("Train/Test split ")
        print(100-test_size*100, "% of training data")
        print(test_size*100, "% of testing data")    

    # split data into train and test sets
    train_examples, test_examples, train_labels, test_labels = train_test_split(
        examples, 
        labels, 
        test_size=0.1, 
        random_state=42, 
        shuffle=True
    )

    # transform train and test examples to their corresponding one-hot representations
    train_users = train_examples[:, 0]
    test_users = test_examples[:, 0]

    train_items = train_examples[:, 1]
    test_items = test_examples[:, 1]

    # Final training and test set
    x_train = np.array(list(zip(train_users, train_items)))
    x_test = np.array(list(zip(test_users, test_items)))

    y_train = train_labels
    y_test = test_labels

    if verbose:
        print()
        print('number of training examples : ', x_train.shape)
        print('number of training labels : ', y_train.shape)
        print('number of test examples : ', x_test.shape)
        print('number of test labels : ', y_test.shape)

    return (x_train, x_test), (y_train, y_test)


def mean_ratings(dataframe):
    means = dataframe.groupby(by='userId', as_index=False)['rating'].mean()
    return means


def normalized_ratings(dataframe, norm_column="norm_rating"):
    """
    Нормализация рейтинга пользователя относительно общего среднего
    """
    mean = mean_ratings(dataframe=dataframe)
    norm = pd.merge(dataframe, mean, suffixes=('', '_mean'), on='userId')
    norm[f'{norm_column}'] = norm['rating'] - norm['rating_mean']

    return norm


def rating_matrix(dataframe, column):
    crosstab = pd.crosstab(dataframe.userId, dataframe.movieId, dataframe[f'{column}'], aggfunc=sum).fillna(0).values
    matrix = csr_matrix(crosstab)
    return matrix


def scale_ratings(dataframe, scaled_column="scaled_rating"):
    dataframe[f"{scaled_column}"] = dataframe.rating / 5.0
    return dataframe


def get_examples(dataframe, labels_column="rating"):
    examples = dataframe[['userId', 'movieId']].values
    labels = dataframe[f'{labels_column}'].values
    return examples, labels

In [5]:
def ids_encoder(ratings):
    """
        Энкодер для более удобной работы
    """
    users = sorted(ratings['userId'].unique())
    items = sorted(ratings['movieId'].unique())

    # энкодер для пользователей и элементов
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit
    uencoder.fit(users)
    iencoder.fit(items)

    # перезапись ID
    ratings.userId = uencoder.transform(ratings.userId.tolist())
    ratings.movieId = iencoder.transform(ratings.movieId.tolist())

    return ratings, uencoder, iencoder

## Модель

In [6]:
class MatrixFactorization:
    
    def __init__(self, m, n, k=10, alpha=0.001, lamb=0.01):
        """
              
        : param
            - m : кол-во пользователей
            - n : кол-во элементов
            - k : длина факторов (для пользователей и элементов)
            - alpha : learning rate 
            - lamb : regularizer
        """
        np.random.seed(32)
        
        # создаем матрицы P / Q
        self.k = k
        self.P = np.random.normal(size=(m, k))
        self.Q = np.random.normal(size=(n, k))
        
        # сохраняем гиперпараметры
        self.alpha = alpha
        self.lamb = lamb
        
        # словарь для сохранения обучения 
        self.history = {
            "epochs":[],
            "loss":[],
            "val_loss":[],
            "lr":[]
        }
    
    def print_training_parameters(self):
        print('Обучаем Matrix Factorization  ...')
        print(f'k={self.k} \t alpha={self.alpha} \t lambda={self.lamb}')
    
    def update_rule(self, u, i, error):
        self.P[u] = self.P[u] + self.alpha * (error * self.Q[i] - self.lamb * self.P[u])
        self.Q[i] = self.Q[i] + self.alpha * (error * self.P[u] - self.lamb * self.Q[i])
        
    def mae(self,  x_train, y_train):
        """
        функция возвращает MAE
        """
        # кол-во в сэплте
        M = x_train.shape[0]
        error = 0
        for pair, r in zip(x_train, y_train):
            u, i = pair
            error += abs(r - np.dot(self.P[u], self.Q[i]))
        return error/M
    
    def print_training_progress(self, epoch, epochs, error, val_error, steps=5):
        if epoch == 1 or epoch % steps == 0 :
                print("epoch {}/{} - loss : {} - val_loss : {}".format(epoch, epochs, round(error,3), round(val_error,3)))
                
    def learning_rate_schedule(self, epoch, target_epochs = 20):
        if (epoch >= target_epochs) and (epoch % target_epochs == 0):
                factor = epoch // target_epochs
                self.alpha = self.alpha * (1 / (factor * 20))
                print("\nLearning Rate : {}\n".format(self.alpha))
    
    def fit(self, x_train, y_train, validation_data, epochs=1000):
        """
        Обучение на факторах P и Q с проверкой через тестовый набор данных
        
        :param
            - x_train : пара для обучения (u,i) где рейтинг известный
            - y_train : набор рейтингов r_ui для пары (u,i)
            - validation_data : tuple (x_test, y_test)
            - epochs : кол-во валидаций
            
        """
        self.print_training_parameters()
        
        # валидация
        x_test, y_test = validation_data
        
        # цикл по эпохам
        for epoch in range(1, epochs+1):
            
            # для каждой пары (u,i) и рейтинга r (который известный)
            for pair, r in zip(x_train, y_train):

                # разкрываем пару значений
                u,i = pair

                # вычисляем предик
                r_hat = np.dot(self.P[u], self.Q[i])

                # считаем ошибку
                e = abs(r - r_hat)

                # обновляем
                self.update_rule(u, i, e)
  
                    
                
            # финализация
            error = self.mae(x_train, y_train)
            val_error = self.mae(x_test, y_test)
            
            # обновление словоря
            self.history['epochs'].append(epoch)
            self.history['loss'].append(error)
            self.history['val_loss'].append(val_error)
            
            # обновление истории
            self.update_history(epoch, error, val_error)
            
            # print
            self.print_training_progress(epoch, epochs, error, val_error, steps=1)
        
        return self.history
    
    def update_history(self, epoch, error, val_error):
        self.history['epochs'].append(epoch)
        self.history['loss'].append(error)
        self.history['val_loss'].append(val_error)
        self.history['lr'].append(self.alpha)
    
    def evaluate(self, x_test, y_test):
        """
        Вычисление глобальной ошибки на тестовой выборке     
        :param x_test : тестовая пара (u,i) 
        :param y_test : рейтинг r_ui для всех пар (u,i)
        """
        error = self.mae(x_test, y_test)
        print(f"validation error : {round(error,3)}")
        
        return error
      
    def predict(self, userid, itemid):
        """
        Предикт для всех пользователей и элементов
        :param userШd
        :param itemId
        :return r : предикт
        """
        
        u = uencoder.transform([userid])[0]
        i = iencoder.transform([itemid])[0]
        
        # вычисление рейтинга
        r = np.dot(self.P[u], self.Q[i])
        return r

    def recommend(self, userid, N=10):
        """
        Топ N рекомендаций для переданного пользователя

        :return(top_items,preds) : Топ N 
        """
        
        u = uencoder.transform([userid])[0]
        
        # предикт
        predictions = np.dot(self.P[u], self.Q.T)

        # индекст Топ N
        # только необходимое кол-во
        top_items = self.iencoder.inverse_transform(top_idx)
        top_idx = np.flip(np.argsort(predictions))[:N]
        preds = predictions[top_idx]

        return top_items, preds        

In [7]:
epochs = 10

### Сдетаем тест над данными

In [9]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

m = ratings.userId.nunique()   # всего пользователей
n = ratings.movieId.nunique()   # всего элементов

ratings, uencoder, iencoder = ids_encoder(ratings)

# получение данных в подготовленном виде
raw_examples, raw_labels = get_examples(ratings)

# train test split
(x_train, x_test), (y_train, y_test) = ttsplit(examples=raw_examples, labels=raw_labels)

In [13]:
# модель
MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)

# fit 
history = MF.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))

Обучаем Matrix Factorization  ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/10 - loss : 3.371 - val_loss : 3.441
epoch 2/10 - loss : 2.745 - val_loss : 2.812
epoch 3/10 - loss : 2.244 - val_loss : 2.313
epoch 4/10 - loss : 2.009 - val_loss : 2.078
epoch 5/10 - loss : 1.882 - val_loss : 1.95
epoch 6/10 - loss : 1.803 - val_loss : 1.871
epoch 7/10 - loss : 1.748 - val_loss : 1.816
epoch 8/10 - loss : 1.708 - val_loss : 1.777
epoch 9/10 - loss : 1.678 - val_loss : 1.747
epoch 10/10 - loss : 1.654 - val_loss : 1.723


In [14]:
MF.evaluate(x_test, y_test)

validation error : 1.723


1.723132441351157

### Нормализованные рейтинги

In [15]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

m = ratings.userId.nunique()   # всего пользователей
n = ratings.movieId.nunique()   # всего элементов

ratings, uencoder, iencoder = ids_encoder(ratings)

# нормализация по среднему
normalized_column_name = "norm_rating"
ratings = normalized_ratings(ratings, norm_column=normalized_column_name)

# подготовленные данные с нормализацией
raw_examples, raw_labels = get_examples(ratings, labels_column=normalized_column_name)

# train test split
(x_train, x_test), (y_train, y_test) = ttsplit(examples=raw_examples, labels=raw_labels)

In [16]:
# модель
MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)

# fit 
history = MF.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))

Обучаем Matrix Factorization  ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/10 - loss : 0.801 - val_loss : 0.79
epoch 2/10 - loss : 0.76 - val_loss : 0.753
epoch 3/10 - loss : 0.752 - val_loss : 0.747
epoch 4/10 - loss : 0.749 - val_loss : 0.745
epoch 5/10 - loss : 0.748 - val_loss : 0.744
epoch 6/10 - loss : 0.747 - val_loss : 0.744
epoch 7/10 - loss : 0.747 - val_loss : 0.744
epoch 8/10 - loss : 0.746 - val_loss : 0.743
epoch 9/10 - loss : 0.746 - val_loss : 0.743
epoch 10/10 - loss : 0.746 - val_loss : 0.743


In [17]:
MF.evaluate(x_test, y_test)

validation error : 0.743


0.7432042455456079

### Предикт

Латентные факторы в матрицах $P$ и $Q$ позволяют создавать предикт рейтингов для элементов

In [18]:
ratings.userid = uencoder.inverse_transform(ratings.userId.to_list())
ratings.itemid = iencoder.inverse_transform(ratings.movieId.to_list())
ratings.head(5)

,userId,movieId,rating,timestamp,rating_mean,norm_rating
0,0,30,2.5,1260759144,2.55,-0.05
1,0,833,3.0,1260759179,2.55,0.45
2,0,859,3.0,1260759182,2.55,0.45
3,0,906,2.0,1260759185,2.55,-0.55
4,0,931,4.0,1260759205,2.55,1.45


In [19]:
4.188679 + MF.predict(userid=1, itemid=1) # добавим средний рейтинг для предикта, т.к. ранее мы нормализовали

4.188451632987444

In [21]:
MF.predict(userid=1, itemid=1)

-0.00022736701255557363